<a href="https://colab.research.google.com/github/letrongminh/MDL4Training/blob/main/ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Conv2D, BatchNormalization, Activation, Add, MaxPool2D
from tensorflow.keras import Model

# **Định nghĩa và khởi tạo Residual Block**

In [2]:
class Identify_block(Model):
  def __init__(self, filters, kernel_size):
    super(Identify_block, self).__init__(name = '')

    self.conv_1 = Conv2D(filters, kernel_size, padding='same')
    self.batch_norm_1 = BatchNormalization()
    
    self.conv_2 = Conv2D(filters, kernel_size, padding='same')
    self.batch_norm_2 = BatchNormalization()

    self.activation = Activation('relu')

    # Add() - nhận đầu vào là các tensor có cùng chiều, trả về một tensor có cùng chiều
    self.add_layer = Add()

  def call(self, input_tensors):
    x = self.conv_1(input_tensors)
    x = self.batch_norm_1(x)
    x = self.activation(x)

    x = self.conv_2(x)
    x = self.batch_norm_2(x)
    
    x = self.add_layer([x, input_tensors])
    x = self.activation(x)

    return x

# **Xây dựng model ResNet**
**sử dụng lại Residual Bock đã tạo**

In [3]:
class ResNet(Model):
  def __init__(self, number_classes):
    super(ResNet, self).__init__()
    self.conv = Conv2D(64, 7, padding='same')
    self.batch_norm = BatchNormalization()
    self.activation = Activation('relu')
    self.max_pool = MaxPool2D((3, 3))

    # Use Identify Block
    self.identify_1 = Identify_block(64, 3)
    self.identify_2 = Identify_block(64, 3)

    self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
    self.classifier = Dense(number_classes, activation='softmax')
    
  def call(self, inputs):
    x = self.conv(inputs)
    x = self.batch_norm(x)
    x = self.activation(x)
    x = self.max_pool(x)

    # block = indentify block
    x = self.identify_1(x)
    x = self.identify_2(x)

    x = self.global_pool(x)
    
    return self.classifier(x)

**Training**

In [4]:
import tensorflow_datasets as tfds

def preprocess(features):
    return tf.cast(features['image'], tf.float32) / 255., features['label']
    
# load and preprocess the dataset
dataset = tfds.load('mnist', split=tfds.Split.TRAIN, data_dir='./data')
dataset = dataset.map(preprocess).batch(32)



# Tune for loss, optimizer
resnet = ResNet(10)
resnet.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics='accuracy')



# train the model.
resnet.fit(dataset, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 27s 13ms/step - loss: 0.3365 - accuracy: 0.9293
Epoch 2/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0969 - accuracy: 0.9763
Epoch 3/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0690 - accuracy: 0.9825
Epoch 4/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0548 - accuracy: 0.9862
Epoch 5/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0456 - accuracy: 0.9883
Epoch 6/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0390 - accuracy: 0.9900
Epoch 7/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0339 - accuracy: 0.9916
Epoch 8/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0299 - accuracy: 0.9927
Epoch 9/10
1875/1875 [==============================] - 18s 9ms/step - loss: 0.0264 - accuracy: 0.9939
Epoch 10/10
1875/1875 [==============================] - 18s 10ms/step